# 基于RAG的论文信息检索-矢量召回

In [1]:
from helper.read_pdf import read_pdf,preprocess
from helper.rag import build_prompt, get_completion
from helper.vecdb import VectorDBConnector
from helper.simcse_embedding import get_embedding

/Users/sunwenjun/anaconda3/envs/python310/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-18 20:38:20.157128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1、读取pdf文档

In [2]:
pdf_filepath = './data/基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究2020.pdf'  # 示例PDF文件路径
text_list = read_pdf(pdf_filepath, min_line_length=5)  # 读取PDF文件中的文本
print("text_list:", len(text_list))

pdf_text_list = preprocess(text_list) # 对每段文本预处理
print("pdf_text_list:", len(pdf_text_list))

text_list: 363


## 2、创建矢量库

In [3]:
vec_db = VectorDBConnector("demo", get_embedding)
vec_db.add_document(pdf_text)

No sentence-transformers model found with name /Users/sunwenjun/data/models_from_hf/princeton-nlp/sup-simcse-bert-base-uncased. Creating a new one with mean pooling.


## 3、Prompt模版

In [4]:
prompt_template = """
已知信息:
__INFO__

用户问：
__QUERY__

请用中文回答用户问题。
"""

## 4、检索

In [5]:
user_query = "LSTM网络是如何应用在地面站天线调度的？"
search_results = vec_db.search(user_query, 10)

for para in search_results['documents'][0]:
    print(para + "\n")

No sentence-transformers model found with name /Users/sunwenjun/data/models_from_hf/princeton-nlp/sup-simcse-bert-base-uncased. Creating a new one with mean pooling.


2 3 LSTM 提取的天线使用特征和地面站现有优先级规则综合得到的天线使用优先

(2)LSTM 提取的天线使用优先级规则,相比于地面站现有天线使用优先 级规则时效性较差

19 第4章 遥感卫星地面站天线智能调度 集用来调整 LSTM 模型参数,验证集用来选择效果最佳的模型

11 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究 短持续时间

V 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究

9 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究 第 3 章 地面站天线资源调度问题

(3) 使用集束搜索的启发式搜索,为每个地面站的数据接收任务确定接收天线

69 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究

57 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究

所示,使用 1减去 LSTM 提取的天线使用概率值,并与地面站 现有优先级规则在数值上对应相乘,得出新的天线使用优先级规则,见附表1、



## 5、根据Prompt模版生成Prompt

In [6]:
prompt = build_prompt(prompt_template, info=search_results['documents'][0], query=user_query)
print("Prompt:\n", prompt)

Prompt:
 
已知信息:
2 3 LSTM 提取的天线使用特征和地面站现有优先级规则综合得到的天线使用优先
(2)LSTM 提取的天线使用优先级规则,相比于地面站现有天线使用优先 级规则时效性较差
19 第4章 遥感卫星地面站天线智能调度 集用来调整 LSTM 模型参数,验证集用来选择效果最佳的模型
11 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究 短持续时间
V 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究
9 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究 第 3 章 地面站天线资源调度问题
(3) 使用集束搜索的启发式搜索,为每个地面站的数据接收任务确定接收天线
69 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究
57 基于LSTM和启发式搜索的遥感卫星地面站天线智能调度方法研究
所示,使用 1减去 LSTM 提取的天线使用概率值,并与地面站 现有优先级规则在数值上对应相乘,得出新的天线使用优先级规则,见附表1、

用户问：
LSTM网络是如何应用在地面站天线调度的？

请用中文回答用户问题。



## 6、调用大模型,参考检索结果，生成问题答案

In [7]:
response = get_completion(prompt)
print("答案：\n", response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


答案：
 LSTM网络在地面站天线调度中的应用主要体现在以下几个方面：

1. **特征提取**：LSTM网络能够提取天线使用的特征，这些特征可能包括天线的性能参数、历史使用情况、任务需求等。

2. **优先级规则生成**：通过LSTM网络提取的特征，可以生成天线使用的优先级规则。这些规则可以帮助地面站更智能地决定哪些天线应该优先使用。

3. **时效性对比**：LSTM提取的天线使用优先级规则与地面站现有的优先级规则相比，可能在时效性上存在差异。这意味着LSTM生成的规则可能需要进一步优化以适应实时调度的需求。

4. **模型训练与验证**：使用训练集来调整LSTM模型的参数，验证集则用于选择效果最佳的模型。这有助于确保LSTM网络在实际应用中的有效性和准确性。

5. **启发式搜索结合**：LSTM网络可以与启发式搜索方法结合使用，通过集束搜索等启发式算法为每个地面站的数据接收任务确定接收天线，提高调度的效率和准确性。

6. **优先级规则调整**：通过将LSTM提取的天线使用概率值进行调整（例如使用1减去概率值），并与地面站现有优先级规则相结合，可以得出新的天线使用优先级规则，以适应不同的调度场景。

综上所述，LSTM网络在地面站天线调度中的应用是通过特征提取、优先级规则生成、模型训练与验证、启发式搜索结合以及优先级规则调整等多个步骤实现的，旨在提高天线调度的智能化水平和效率。
